<a href="https://colab.research.google.com/github/ketanp23/sit-neuralnetworks-class/blob/main/CNN_Example_(MNIST).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import ssl
import time

# Temporarily disable SSL verification for dataset download if needed
# This helps in environments where certificate verification fails.
ssl._create_default_https_context = ssl._create_unverified_context

# --- 1. Define the CNN Architecture ---
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 1 input channel (grayscale image), 6 output channels, 5x5 kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        # 6 input channels, 16 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)

        # Fully connected layers (The input size is calculated based on the image dimensions
        # after two pooling layers: 28x28 -> (28-4)/2 = 12x12 -> (12-4)/2 = 4x4)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 output classes (digits 0-9)

    def forward(self, x):
        # -> Convolution -> ReLU -> Pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # -> Convolution -> ReLU -> Pooling
        x = self.pool(torch.relu(self.conv2(x)))

        # Flatten all dimensions except batch
        x = torch.flatten(x, 1)

        # -> Fully Connected Layers -> ReLU
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        # -> Output Layer
        x = self.fc3(x)
        return x

# --- 2. Data Loading and Transformation ---

# Define transformation: convert to Tensor and normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # Standard MNIST normalization
])

# Download and load training dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load test dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)

# --- 3. Initialize Model, Loss, and Optimizer ---
net = Net()
# Use CrossEntropyLoss for multi-class classification
criterion = nn.CrossEntropyLoss()
# Use Stochastic Gradient Descent (SGD) optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("--- Starting CNN Training on MNIST ---")
start_time = time.time()

# --- 4. Training Loop (2 Epochs) ---
NUM_EPOCHS = 2

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] running loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print(f"\nTraining finished in {time.time() - start_time:.2f} seconds.")
print("-" * 40)

# --- 5. Testing the Model (Generalization Check) ---
correct = 0
total = 0
total_test_loss = 0

# Set model to evaluation mode
net.eval()

# Since we don't need to compute gradients during testing, we wrap it in torch.no_grad()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)

        # Calculate loss on test set
        test_loss = criterion(outputs, labels)
        total_test_loss += test_loss.item()

        # Get the predicted class (max log-probability)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# --- 6. Print Output ---
final_accuracy = 100 * correct / total
avg_test_loss = total_test_loss / len(testloader)

print("--- CNN Performance Metrics ---")
print(f'Total Test Images: {total}')
print(f'Accuracy of the network on the 10000 test images: {final_accuracy:.2f} %')
print(f'Average Test Loss: {avg_test_loss:.4f}')

# Example: Print a quick per-class accuracy check
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print("\nAccuracy per class:")
for i in range(10):
    if class_total[i] > 0:
        print(f'  Accuracy of {i}: {100 * class_correct[i] / class_total[i]:.1f} %')

100%|██████████| 9.91M/9.91M [00:00<00:00, 22.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 599kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 5.55MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.7MB/s]


--- Starting CNN Training on MNIST ---
[Epoch 1, Batch   200] running loss: 2.275
[Epoch 1, Batch   400] running loss: 1.835
[Epoch 1, Batch   600] running loss: 0.623
[Epoch 1, Batch   800] running loss: 0.390
[Epoch 2, Batch   200] running loss: 0.286
[Epoch 2, Batch   400] running loss: 0.236
[Epoch 2, Batch   600] running loss: 0.209
[Epoch 2, Batch   800] running loss: 0.190

Training finished in 57.20 seconds.
----------------------------------------
--- CNN Performance Metrics ---
Total Test Images: 10000
Accuracy of the network on the 10000 test images: 94.81 %
Average Test Loss: 0.1679

Accuracy per class:
  Accuracy of 0: 98.1 %
  Accuracy of 1: 97.0 %
  Accuracy of 2: 92.9 %
  Accuracy of 3: 94.0 %
  Accuracy of 4: 97.0 %
  Accuracy of 5: 95.4 %
  Accuracy of 6: 97.5 %
  Accuracy of 7: 90.9 %
  Accuracy of 8: 97.6 %
  Accuracy of 9: 88.0 %
